In [4]:
from neo4j import GraphDatabase
from dotenv import load_dotenv
import os
import re

load_dotenv()

NEO4J_URI = os.getenv("NEO4J_URI")
NEO4J_USERNAME = os.getenv("NEO4J_USERNAME")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")

In [3]:
class BirthdayImporter:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self.driver.close()

    def create_birthday_node(self, name, birthday):
        if self._validate_birthday_format(birthday):
            with self.driver.session() as session:
                session.write_transaction(self._create_node, name, birthday)
        else:
            print(f"Ignoring invalid birthday format for {name}: {birthday} (please use this format : 'DD-MM-YYYY' ignoring 0 at first position).")

    @staticmethod
    def _validate_birthday_format(birthday):
        pattern = re.compile(r'^\d{1,2}-\d{1,2}-\d{4}$')
        return bool(pattern.match(birthday))

    @staticmethod
    def _create_node(tx, name, birthday):
        query = (
            "CREATE (p:Family {name: $name, birthday: $birthday}) RETURN p"
        )
        return tx.run(query, name=name, birthday=birthday)

def load_birthdays(file_path, uri, user, password):
    importer = BirthdayImporter(uri, user, password)
    try:
        with open(file_path, 'r') as file:
            for line in file:
                name, birthday = line.strip().split(', ')
                importer.create_birthday_node(name, birthday)
    finally:
        importer.close()

In [ ]:
load_birthdays("birthdays.txt", NEO4J_URI, NEO4J_USERNAME, NEO4J_PASSWORD)